# <p style="text-align:center;"><u>__EDA - Exploratory Data Analysis__<u></p>

### **🧰 Dependencies**

 `import pandas as pd`
Imports the **pandas** library, a powerful tool for data manipulation and analysis using DataFrames.

 `import numpy as np`
Imports **NumPy**, a fundamental library for numerical computing in Python — often used for handling arrays, math operations, and missing values.


In [ ]:
import pandas as pd
import numpy as np

# 🚄 SNCF Data Cleaner - Railway Data Cleaning Toolkit 🧹

*Your all-in-one solution for cleaning and preprocessing French railway data with style and precision.*

---

## 🔍 Overview

This module offers a robust set of specialized functions to clean and prepare operational data from the French national railway system (SNCF). From removing duplicates to correcting 
station names, it’s built to address real-world data quality issues commonly found in transportation datasets.


---

## 📋 Core Functions

### 🧽 Basic Cleaning

| Function                                | Description                                                                         |
| --------------------------------------- | ----------------------------------------------------------------------------------- |
| `remove_duplicates(df)`                 | Removes duplicate rows — because a train can't be in two places at once!            |
| `remove_negative(df)`                   | Filters out impossible negative values from key metrics (e.g., -3 cancelled trains) |
| `fill_missed_value(df)`                 | Smart imputation: uses median for numeric columns and preserves NaNs for text       |
| `missing_mandatory(df, mandatory_cols)` | Drops rows with missing or invalid values in critical fields                        |

### 📅 Date Handling

| Function              | Description                                               |
| --------------------- | --------------------------------------------------------- |
| `date_formatting(df)` | Converts and standardizes all dates to `'YYYY-MM'` format |

### 🏷️ Station and Service Name Correction

| Function                                                                                 | Description                                                                                                       |
| ---------------------------------------------------------------------------------------- | ----------------------------------------------------------------------------------------------------------------- |
| `correct_text_columns(df, station_columns, service_columns, station_list, service_list)` | Uses fuzzy matching (Levenshtein distance) to fix typos in station and service names, with caching for efficiency |
#
---

## 🛠 Technical Details

**Key Features:**

* Custom implementation of the Levenshtein distance for flexible fuzzy matching
* Caching system to speed up repeated corrections
* Configurable similarity thresholds (default: 0.7)

**Predefined Station Names:**
Includes over 50 French and international stations, such as:

* **Major hubs**: Paris Montparnasse, Lyon Part-Dieu, Marseille St Charles
* **Cross-border links**: Geneva, Barcelona, Zurich
* **Regional stops**: Quimper, Tours, Angoulême

**Service Types:**

* `National`
* `International`
#
---
#
## ⚙️ Dependencies

```bash
pip install pandas numpy
```

In [9]:

correct_stations = [
    "BORDEAUX ST JEAN", "LA ROCHELLE VILLE", "PARIS MONTPARNASSE", "QUIMPER", "TOURS",
    "ST PIERRE DES CORPS", "ST MALO", "NANTES", "PARIS EST", "STRASBOURG", "DUNKERQUE",
    "LILLE", "PARIS VAUGIRARD", "RENNES", "TOURCOING", "CHAMBERY CHALLES LES EAUX",
    "LYON PART DIEU", "MONTPELLIER", "MULHOUSE VILLE", "NICE VILLE", "PARIS LYON",
    "BARCELONA", "GENEVE", "MADRID", "BREST", "POITIERS", "TOULOUSE MATABIAU",
    "MARNE LA VALLEE", "MARSEILLE ST CHARLES", "FRANCFORT", "ANGOULEME", "METZ",
    "PARIS NORD", "BELLEGARDE (AIN)", "MACON LOCHE", "PERPIGNAN", "DOUAI",
    "VALENCE ALIXAN TGV", "LAUSANNE", "ANGERS SAINT LAUD", "STUTTGART", "LAVAL",
    "NANCY", "BESANCON FRANCHE COMTE TGV", "GRENOBLE", "NIMES", "SAINT ETIENNE CHATEAUCREUX",
    "ITALIE", "ZURICH", "VANNES", "ANNECY", "AVIGNON TGV", "LE MANS", "ARRAS",
    "DIJON VILLE", "LE CREUSOT MONTCEAU MONTCHANIN", "REIMS"
]

correct_services = ["National", "International"]

mandatory_cols = ["Date", "Service", "Departure station", "Arrival station"]

station_cols = ["Departure station", "Arrival station"]

service_cols = ["Service"]

def remove_duplicates(df):
    df = df.drop_duplicates()
    df = df.drop_duplicates(subset=mandatory_cols, keep='first')
    return df.reset_index(drop=True)

def remove_negative(df):
    cols_num = [
        'Average journey time',
        'Number of scheduled trains',
        'Number of cancelled trains',
        'Number of trains delayed at departure',
        'Number of trains delayed at arrival'
    ]
    condition = (df[cols_num] >= 0).all(axis=1)
    return df[condition]

def fill_missed_value(df):
    # cols num
    num_cols = df.select_dtypes(include=['number']).columns
    for col in num_cols:
        if df[col].dropna().empty:
            df[col] = df[col].fillna(0)
        else:
            df[col] = df[col].fillna(df[col].median())
    # cols str
    str_cols = df.select_dtypes(include=['object']).columns
    for col in str_cols:
        df[col] = df[col].fillna(np.nan)
    return df

def date_formatting(df):
    df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m", errors="coerce")
    df["Date"] = df["Date"].dt.strftime("%Y-%m")
    return df

def my_levenshtein(a, b, ratio=False, print_matrix=False, lowercase=False):
    if type(a) is not str:
        raise TypeError('First argument is not a string!')
    if type(b) is not str:
        raise TypeError('Second argument is not a string!')
    if a == '':
        return len(b)
    if b == '':
        return len(a)
    if lowercase:
        a = a.lower()
        b = b.lower()
    n = len(a)
    m = len(b)
    lev = np.zeros((n+1, m+1), dtype=np.uint16)
    for i in range(n+1):
        lev[i, 0] = i
    for j in range(m+1):
        lev[0, j] = j
    for i in range(1, n+1):
        for j in range(1, m+1):
            insertion = lev[i-1, j] + 1
            deletion = lev[i, j-1] + 1
            substitution = lev[i-1, j-1] + (1 if a[i-1] != b[j-1] else 0)
            lev[i, j] = min(insertion, deletion, substitution)
    if print_matrix:
        print(lev)
    if ratio:
        return (n + m - lev[n, m]) / (n + m)
    else:
        return lev[n, m]

def missing_mandatory(df, mandatory_cols):
    df_clean = df.copy()
    for col in mandatory_cols:
        if col in df_clean.columns:
            idx_to_drop = df_clean[
                df_clean[col].isna() | (df_clean[col].astype(str).str.lower() == 'nan')
            ].index
            df_clean = df_clean.drop(index=idx_to_drop)
    return df_clean

def find_closest_station(station_name, correct_stations, threshold=0.7):
    station_name = station_name.upper()
    best_match = station_name
    best_score = 0
    for correct_station in correct_stations:
        score = my_levenshtein(station_name, correct_station.upper(), ratio=True)
        if score > best_score:
            best_score = score
            best_match = correct_station
    return best_match if best_score >= threshold else station_name

def find_closest_service(service_name, correct_services, threshold=0.7):
    best_match = service_name
    best_score = 0
    for correct_service in correct_services:
        score = my_levenshtein(service_name, correct_service, ratio=True)
        if score > best_score:
            best_score = score
            best_match = correct_service
    return best_match if best_score >= threshold else service_name

def correct_text_columns(df, station_columns, services_columns, station_list, services_list):
    df_corrected = df.copy()
    station_cache = {}
    service_cache = {}

    for col in station_columns:
        if col in df_corrected.columns:
            def get_corrected_station_name(station_name):
                station_str = str(station_name).strip().upper()
                if station_str in station_cache:
                    return station_cache[station_str]
                corrected = find_closest_station(station_str, station_list)
                station_cache[station_str] = corrected
                return corrected
            df_corrected[col] = df_corrected[col].apply(get_corrected_station_name)

    for col in services_columns:
        if col in df_corrected.columns:
            def get_corrected_service_name(service_name):
                service_str = str(service_name).strip()
                if service_str in service_cache:
                    return service_cache[service_str]
                corrected = find_closest_service(service_str, services_list)
                service_cache[service_str] = corrected
                return corrected
            df_corrected[col] = df_corrected[col].apply(get_corrected_service_name)

    return df_corrected


### **📄 CSV File Handling Functions**

 `read_csv()`
Reads a CSV file named `dataset.csv` using a semicolon (`;`) as the separator.
Returns a `pandas` DataFrame containing the data from the file.

 `to_csv(df)`
Writes a `pandas` DataFrame to a CSV file named `cleaned_dataset.csv` without including the index.
Useful for saving cleaned or processed data.


In [10]:
def to_csv(df):
    df.to_csv('cleaned_dataset.csv', index=False)

def read_csv():
    return pd.read_csv("dataset.csv", sep=';')

### **🧠 Data Processing Pipeline**

`main()`

Executes the complete SNCF data cleaning pipeline in the following order:

1. **Reads** the raw dataset from `dataset.csv` using `read_csv()`.
2. **Removes duplicate rows** with `remove_duplicates(df)`.
3. **Filters out invalid negative values** using `remove_negative(df)`.
4. **Corrects misspelled station and service names** via `correct_text_columns(...)`.
5. **Fills missing values** with appropriate strategies using `fill_missed_value(df)`.
6. **Formats date values** to the `'YYYY-MM'` format using `date_formatting(df)`.
7. **Removes rows with missing mandatory fields** using `missing_mandatory(df, mandatory_cols)`.
8. **Writes** the cleaned dataset to `cleaned_dataset.csv` with `to_csv(df)`.

This function serves as the entry point of the script and runs automatically when the file is executed directly.


In [11]:
def main():
    df = read_csv()
    df = remove_duplicates(df)
    df = remove_negative(df)
    df = correct_text_columns(df, station_cols, service_cols, correct_stations, correct_services)
    df = fill_missed_value(df)
    df = date_formatting(df)
    df = missing_mandatory(df, mandatory_cols)
    to_csv(df)

if __name__ == "__main__":
    main()